In [ ]:
# !pip install PytorchCML

In [2]:

import sys
sys.path.append("../../src/")

from itertools import product

from PytorchCML import losses, models, samplers, evaluators, trainers
import torch
from torch import nn, optim
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
movielens = pd.read_csv(
  'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
  sep='\t', header=None, index_col=None,
)
movielens.columns = ["user_id", "item_id", "rating", "timestamp"]
movielens.user_id -= 1
movielens.item_id -= 1
movielens.rating = (movielens >= 4).astype(int)
n_user = movielens.user_id.nunique()
n_item = movielens.item_id.nunique()

train, test = train_test_split(movielens)


# all user item pairs
df_all = pd.DataFrame(
    [[u, i] for u,i in product(range(n_user), range(n_item))],
    columns=["user_id", "item_id"]
)

# frag train pairs
df_all = pd.merge(
    df_all, 
    train[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
)

# remove train pairs
test = pd.merge(
    df_all[df_all.rating.isna()][["user_id", "item_id"]], 
    test[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
).fillna(0)

# numpy array
train_set = train[train.rating == 1][["user_id", "item_id"]].values
test_set = test[["user_id", "item_id", "rating"]].values


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, device=device)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(torch.LongTensor(test_set).to(device), score_function_dict, ks=[3,5])
trainer = trainers.CMLTrainer(model, optimizer, criterion, sampler)


In [5]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:37<00:00, 24.90it/s]


In [6]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.014991,0.021031,0.001748,0.015392,0.021031,0.002650,0,NaN
0,0.032581,0.050266,0.002588,0.034547,0.050266,0.004647,10,1405.806201
0,0.114106,0.093020,0.013257,0.114185,0.093020,0.022119,20,973.621241
